## 은성반주기 크롤러

In [7]:
import requests
from bs4 import BeautifulSoup
import time
import lxml

In [8]:
def crawl_music_list(page=2):
    url = f"http://walkmedia.com/board/bbs/board.php?bo_table=music_list&page={page}"
    headers = {
        "User-Agent": ("Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
                       "AppleWebKit/537.36 (KHTML, like Gecko) "
                       "Chrome/112.0.0.0 Safari/537.36")
    }
    resp = requests.get(url, headers=headers)
    if resp.status_code != 200:
        print(f"[ERROR] {page} 페이지 응답 실패:", resp.status_code)
        return []

    # 파서를 lxml로 변경
    soup = BeautifulSoup(resp.text, "lxml")

    # (디버깅) 실제 파싱된 HTML 확인
    print(soup.prettify())

    table = soup.find("table", class_="board_list")
    if not table:
        print("[ERROR] <table class='board_list'> 못 찾음")
        return []

    # table 안에 <tbody>가 분명 있으면 찾을 텐데, 못 찾으면 문법 에러 가능성
    tbody = table.find("tbody")
    if not tbody:
        print("[ERROR] <tbody> 못 찾음")
        return []

    rows = tbody.find_all("tr", {"class": ["bg0", "bg1"]})
    if not rows:
        print("[INFO] 데이터 행이 없음")
        return []

    results = []
    for row in rows:
        cols = row.find_all("td")
        if len(cols) < 9:
            continue
        music_no     = cols[0].get_text(strip=True)
        category     = cols[1].get_text(strip=True)
        title        = cols[2].get_text(strip=True)
        singer       = cols[3].get_text(strip=True)
        first_phrase = cols[4].get_text(strip=True)
        lyricist     = cols[5].get_text(strip=True)
        composer     = cols[6].get_text(strip=True)
        arranger     = cols[7].get_text(strip=True)
        etc          = cols[8].get_text(strip=True)

        results.append({
            "곡번호": music_no,
            "분류": category,
            "곡명": title,
            "가수명": singer,
            "첫소절": first_phrase,
            "작사": lyricist,
            "작곡": composer,
            "편곡": arranger,
            "기타사항1": etc
        })

    return results

if __name__ == "__main__":
    data = crawl_music_list(page=2)
    print("크롤링된 개수:", len(data))
    for item in data:
        print(item)

<!-- <!DOCTYPE HTML PUBLIC "-//W3C//DTD HTML 4.01 Transitional//EN"> -->
<html>
 <head>
  <meta content="BlendTrans(Duration=0.5)" http-equiv="Page-Enter"/>
  <meta content="BlendTrans(Duration=0.5)" http-equiv="Page-exit"/>
  <meta content="text/html; charset=utf-8" http-equiv="content-type"/>
  <title>
   walk &gt; 수록곡검색 2 페이지
  </title>
  <link href="../style.css" rel="stylesheet" type="text/css"/>
 </head>
 <script type="text/javascript">
  // 자바스크립트에서 사용하는 전역변수 선언
var g4_path      = "..";
var g4_bbs       = "bbs";
var g4_bbs_img   = "img";
var g4_url       = "http://walkmedia.com/board";
var g4_is_member = "";
var g4_is_admin  = "";
var g4_bo_table  = "music_list";
var g4_sca       = "";
var g4_charset   = "euc-kr";
var g4_cookie_domain = "";
var g4_is_gecko  = navigator.userAgent.toLowerCase().indexOf("gecko") != -1;
var g4_is_ie     = navigator.userAgent.toLowerCase().indexOf("msie") != -1;
 </script>
 <script src="../js/jquery-1.4.2.min.js" type="text/javascript">
 </script>
 <

In [11]:
import requests
from bs4 import BeautifulSoup
import time

def crawl_music_list(page=2):
    """
    해당 페이지에서 board_list 클래스를 가진 테이블을 찾아
    <tr>들의 데이터를 파싱하는 예시 함수
    """
    # 예시 URL
    base_url = "http://walkmedia.com/board/bbs/board.php?bo_table=music_list&page={}"
    url = base_url.format(page)

    headers = {
        "User-Agent": ("Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
                       "AppleWebKit/537.36 (KHTML, like Gecko) "
                       "Chrome/112.0.0.0 Safari/537.36")
    }
    response = requests.get(url, headers=headers)
    if response.status_code != 200:
        print("[ERROR] 응답 코드:", response.status_code)
        return []

    # 필요하다면 lxml 파서 사용 (pip install lxml)
    soup = BeautifulSoup(response.text, "html.parser")

    # 1) 테이블(class="board_list")을 직접 찾기
    table = soup.find("table", class_="board_list")
    if not table:
        print("[ERROR] <table class='board_list'>를 찾지 못했습니다.")
        return []

    # 2) 테이블 안의 모든 <tr> 태그
    rows = table.find_all("tr")
    if not rows:
        print("[INFO] 테이블 내부에 <tr>가 없습니다.")
        return []

    # 첫 번째 tr은 (곡번호, 분류...) 헤더일 가능성이 큼
    # 나머지가 실제 데이터 행
    data_rows = rows[1:]

    results = []
    for row in data_rows:
        # 각각의 <td>들
        cols = row.find_all("td")
        # 보통 9개 (곡번호, 분류, 곡명, 가수명, 첫소절, 작사, 작곡, 편곡, 기타사항1)
        if len(cols) < 9:
            continue

        # strip=True로 &nbsp; 제거
        music_no     = cols[0].get_text(strip=True)
        category     = cols[1].get_text(strip=True)
        title        = cols[2].get_text(strip=True)
        singer       = cols[3].get_text(strip=True)
        first_phrase = cols[4].get_text(strip=True)
        lyricist     = cols[5].get_text(strip=True)
        composer     = cols[6].get_text(strip=True)
        arranger     = cols[7].get_text(strip=True)
        etc          = cols[8].get_text(strip=True)

        data_dict = {
            "곡번호": music_no,
            "분류": category,
            "곡명": title,
            "가수명": singer,
            "첫소절": first_phrase,
            "작사": lyricist,
            "작곡": composer,
            "편곡": arranger,
            "기타사항1": etc
        }
        results.append(data_dict)

    return results


if __name__ == "__main__":
    data = crawl_music_list(page=2)
    print(f"[INFO] 크롤링된 데이터 개수: {len(data)}")
    for row in data:
        print(row)


[INFO] 크롤링된 데이터 개수: 10
{'곡번호': '11', '분류': '가요', '곡명': '기억속의멜로디', '가수명': '오태호', '첫소절': '기옥속의멜로디나를', '작사': '오태호', '작곡': '오태호', '편곡': '오태호', '기타사항1': ''}
{'곡번호': '12', '분류': '가요', '곡명': '오래전그날(원곡)', '가수명': '윤종신', '첫소절': '교복을벗고처음으로', '작사': '박주연', '작곡': '윤종신.정석원', '편곡': '정석원.유희열', '기타사항1': ''}
{'곡번호': '13', '분류': '가요', '곡명': '파일럿(드라마파일럿OST)', '가수명': '정연준', '첫소절': '그것이끝이라고우린', '작사': '이순자', '작곡': '윤상', '편곡': '윤상', '기타사항1': '드라마 OST'}
{'곡번호': '14', '분류': '가요', '곡명': '약속된이별', '가수명': '박정운', '첫소절': '거리에그려진그림자', '작사': '박정운', '작곡': '박정운', '편곡': '', '기타사항1': ''}
{'곡번호': '15', '분류': '가요', '곡명': '어려워정말', '가수명': '닥터레게', '첫소절': '어려워정말오오오오오', '작사': '김장윤', '작곡': '김장윤', '편곡': '김장윤', '기타사항1': ''}
{'곡번호': '16', '분류': '가요', '곡명': '가을빛추억', '가수명': '신승훈', '첫소절': '스쳐가는비바람에', '작사': '유정연', '작곡': '유정연', '편곡': '유정연.김형석', '기타사항1': ''}
{'곡번호': '17', '분류': '가요', '곡명': '먼그대', '가수명': '이세훈', '첫소절': '오늘밤잠못드는사연', '작사': '양인자', '작곡': '김희갑', '편곡': '김영철', '기타사항1': ''}
{'곡번호': '18', '분류': '가요', '곡명': '당신은사파이어처럼', '가수명': '신승훈', '

말씀하신 "body > table:nth-of-type(2) > tbody > tr > td > table:nth-of-type(2) > tbody > tr > td > form > table > tbody" 라는 CSS Selector를 그대로 쓰면,
실제 페이지 구조에서 “2번째 table”, “그 안의 2번째 table” 등을 정확히 찾아 들어가야 해서,
작은 차이(예: 중간에 보이지 않는 <table>이 하나 끼어 있다거나, 자동 삽입된 <tbody>가 있는 등)만 있어도 None이 반환될 수 있습니다.

하지만, 코드를 보니 실제 데이터를 담고 있는 테이블에 class="board_list" 라는 속성이 붙어 있습니다.
이처럼 고유한 class가 있다면, **굳이 nth-of-type**을 타고 들어가는 대신, class="board_list" 테이블을 바로 찾는 편이 훨씬 안전하고 간단합니다.

아래는 그런 식으로 **table.board_list**만 찾아서, 그 안의 <tr>들을 순회하는 방법입니다.

이렇게 “2번째 테이블” 안에 “2번째 테이블” 안에 또 “<form> 안의 테이블”…” 순서를 **정확히** 캐치하기 어려우니, **중간에 예상치 못한 <table>`**가 추가되면 모든 인덱스가 어긋나버립니다.

CSS의 nth-of-type는 “형제 중에서 같은 태그 명(table)에 대해 몇 번째인가”를 기준으로 계산합니다.
<tbody> 자동 삽입, 잘못된 HTML 문법 등으로도 변수가 많습니다.
반면에 class="board_list" 처럼 명확한 식별자가 있으면,
“이 테이블이 딱 우리가 원하는 데이터 표구나” 하고 바로 찾을 수 있습니다.
그래서 크롤링할 때는 가능하면 id, class, name, data-* 속성 등 고유 Selectors를 쓰는 것이 안정적입니다.

결론
주신 HTML에 **class="board_list"**가 있으므로, 그걸 기준으로 테이블을 찾은 뒤 <tr>를 파싱하는 게 훨씬 쉽고 안정적.
nth-of-type로 시도해서 계속 [ERROR] tbody를 찾지 못했습니다. 문제가 뜬다면, 테이블의 실제 순서가 바뀌었거나, 자동 생성된 <tbody> 문제 등이 있기 때문입니다.
최종적으로는, table.board_list → tr → td 구조를 따라가면 무리 없이 원하는 데이터를 크롤링할 수 있을 것입니다.

In [15]:
import requests
from bs4 import BeautifulSoup
import time

def crawl_music_list(page=1):
    """
    해당 페이지에서 board_list 클래스를 가진 테이블을 찾아
    <tr>들의 데이터를 파싱하는 예시 함수
    """
    # 예시 URL
    base_url = "http://walkmedia.com/board/bbs/board.php?bo_table=music_list&page={}"
    url = base_url.format(page)

    headers = {
        "User-Agent": ("Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
                       "AppleWebKit/537.36 (KHTML, like Gecko) "
                       "Chrome/112.0.0.0 Safari/537.36")
    }
    response = requests.get(url, headers=headers)
    if response.status_code != 200:
        print("[ERROR] 응답 코드:", response.status_code)
        return []

    # 필요하다면 lxml 파서 사용 (pip install lxml)
    soup = BeautifulSoup(response.text, "html.parser")

    # 1) 테이블(class="board_list")을 직접 찾기
    table = soup.find("table", class_="board_list")
    if not table:
        print("[ERROR] <table class='board_list'>를 찾지 못했습니다.")
        return []

    # 2) 테이블 안의 모든 <tr> 태그
    rows = table.find_all("tr")
    if not rows:
        print("[INFO] 테이블 내부에 <tr>가 없습니다.")
        return []

    # 첫 번째 tr은 (곡번호, 분류...) 헤더일 가능성이 큼
    # 나머지가 실제 데이터 행
    data_rows = rows[1:]

    results = []
    for row in data_rows:
        # 각각의 <td>들
        cols = row.find_all("td")
        # 보통 9개 (곡번호, 분류, 곡명, 가수명, 첫소절, 작사, 작곡, 편곡, 기타사항1)
        if len(cols) < 9:
            continue

        # strip=True로 &nbsp; 제거
        music_no     = cols[0].get_text(strip=True)
        category     = cols[1].get_text(strip=True)
        title        = cols[2].get_text(strip=True)
        singer       = cols[3].get_text(strip=True)
        first_phrase = cols[4].get_text(strip=True)
        lyricist     = cols[5].get_text(strip=True)
        composer     = cols[6].get_text(strip=True)
        arranger     = cols[7].get_text(strip=True)
        etc          = cols[8].get_text(strip=True)

        data_dict = {
            "곡번호": music_no,
            "분류": category,
            "곡명": title,
            "가수명": singer,
            "첫소절": first_phrase,
            "작사": lyricist,
            "작곡": composer,
            "편곡": arranger,
            "기타사항1": etc
        }
        results.append(data_dict)

    return results


if __name__ == "__main__":
    data = crawl_music_list(page=1)
    print(f"[INFO] 크롤링된 데이터 개수: {len(data)}")
    for row in data:
        print(row)


[INFO] 크롤링된 데이터 개수: 10
{'곡번호': '1', '분류': '가요', '곡명': '잠못드는밤비는내리고', '가수명': '김건모', '첫소절': '슬픈노래는듣고싶지않아', '작사': '김창환.박광현', '작곡': '김창환.박광현', '편곡': '김건모', '기타사항1': ''}
{'곡번호': '2', '분류': '가요', '곡명': '아껴둔사랑을위해', '가수명': '이주원', '첫소절': '기다려내몸을둘러싼', '작사': '박주연', '작곡': '손무현', '편곡': '손무현', '기타사항1': ''}
{'곡번호': '3', '분류': '가요', '곡명': '마마보이', '가수명': '김준선', '첫소절': '아하우와아하우와', '작사': '김준선', '작곡': '김준선', '편곡': '서재형', '기타사항1': ''}
{'곡번호': '4', '분류': '가요', '곡명': '난단지나일뿐', '가수명': '미스터투(MR. 2)', '첫소절': '세상에서내가제일', '작사': '강은경', '작곡': '윤일상', '편곡': '윤일상', '기타사항1': ''}
{'곡번호': '5', '분류': '가요', '곡명': '그냥걸었어(원곡)', '가수명': '임종환', '첫소절': '처음엔그냥걸었어', '작사': '김준기', '작곡': '김준기', '편곡': '황인경', '기타사항1': ''}
{'곡번호': '6', '분류': '가요', '곡명': '넌언제나(원곡)', '가수명': '모노', '첫소절': '하루하루늘어갈뿐이야', '작사': '장경아', '작곡': '박정원', '편곡': '박정원', '기타사항1': ''}
{'곡번호': '7', '분류': '가요', '곡명': '너는왜(원곡)', '가수명': '철이와미애', '첫소절': '오오오오오오오오', '작사': '이승호', '작곡': '김진', '편곡': '김진', '기타사항1': ''}
{'곡번호': '8', '분류': '가요', '곡명': '비창(원곡)', '가수명': '이상우', '첫소절': '나

In [22]:
def crawl_music_list(page=1):
    """
    지정된 페이지 번호(page)에 대해
    http://walkmedia.com/board/bbs/board.php?bo_table=music_list&page=page
    에 있는 곡 정보(트)를 크롤링하고,
    [ {곡번호, 분류, 곡명, ...}, ... ] 형태로 반환.
    """
    base_url = "http://walkmedia.com/board/bbs/board.php?bo_table=music_list&page={}"
    url = base_url.format(page)

    headers = {
        "User-Agent": ("Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
                       "AppleWebKit/537.36 (KHTML, like Gecko) "
                       "Chrome/112.0.0.0 Safari/537.36")
    }
    resp = requests.get(url, headers=headers)
    if resp.status_code != 200:
        print(f"[ERROR] {page} 페이지 요청 실패: {resp.status_code}")
        return []

    soup = BeautifulSoup(resp.text, "html.parser")

    # board_list 클래스를 가진 테이블 찾기
    table = soup.find("table", class_="board_list")
    if not table:
        print(f"[WARN] {page} 페이지에서 <table class='board_list'>를 찾지 못했습니다.")
        return []

    # 테이블 안의 <tr> 태그 전체
    rows = table.find_all("tr")
    if not rows:
        print(f"[INFO] {page} 페이지에서 <tr>가 없습니다.")
        return []

    # 첫 번째 <tr>는 헤더, 그 이후부터 실제 데이터로 간주
    data_rows = rows[1:]

    results = []
    for row in data_rows:
        cols = row.find_all("td")
        if len(cols) < 9:
            # 컬럼이 9개 미만이면(헤더거나 이상한 행), 넘어가기
            continue

        music_no     = cols[0].get_text(strip=True)
        category     = cols[1].get_text(strip=True)
        title        = cols[2].get_text(strip=True)
        singer       = cols[3].get_text(strip=True)
        first_phrase = cols[4].get_text(strip=True)
        lyricist     = cols[5].get_text(strip=True)
        composer     = cols[6].get_text(strip=True)
        arranger     = cols[7].get_text(strip=True)
        etc          = cols[8].get_text(strip=True)

        data_dict = {
            "곡번호": music_no,
            "분류": category,
            "곡명": title,
            "가수명": singer,
            "첫소절": first_phrase,
            "작사": lyricist,
            "작곡": composer,
            "편곡": arranger,
            "기타사항1": etc
        }
        results.append(data_dict)

    return results

In [23]:


def get_last_page_number():
    base_url = "http://walkmedia.com/board/bbs/board.php?bo_table=music_list&page=1"
    resp = requests.get(base_url)
    soup = BeautifulSoup(resp.text, "html.parser")

    # "맨끝" 아이콘 찾아서 a태그의 href 파싱
    end_page_link = soup.select_one('div.board_page a img[title="맨끝"]')
    if not end_page_link:
        print("[WARN] 맨끝 페이지 링크를 찾지 못했습니다. 기본값(5584) 사용.")
        return 5584

    # img 태그 말고 그 상위 <a> 태그의 href를 얻어야 함
    a_tag = end_page_link.parent  # parent가 <a> 일 것
    href = a_tag.get("href", "")
    # href 예) "./board.php?bo_table=music_list&page=2&page=5584"

    # parse_qs 쓰면 ? 뒤의 쿼리 파라미터를 dict로 바꿀 수 있음
    # 다만 ./board.php 처럼 상대경로일 경우를 대비해 urljoin을 쓸 수도 있음
    full_url = urljoin(base_url, href)
    parsed = urlparse(full_url)
    qs = parse_qs(parsed.query)
    # qs => {'bo_table': ['music_list'], 'page': ['5584']} 이런 식 or {'page': ['2','5584']}

    # 'page'가 여러 개 있을 수도 있으니 가장 큰 값 찾기
    # 또는 링크 구조를 보고 적절히 처리
    pages = qs.get("page", [])
    page_numbers = [int(p) for p in pages if p.isdigit()]

    if not page_numbers:
        print("[WARN] page 파라미터가 없어서 기본값 5584 사용.")
        return 5584

    return max(page_numbers)

In [ ]:
def crawl_all_pages(start_page=1, end_page=2):
    """
    start_page부터 end_page까지(기본은 1~5584),
    각 페이지를 순회하며 crawl_music_list를 호출.
    모든 데이터를 하나의 리스트로 모아서 반환.
    """
    all_data = []
    for p in range(start_page, end_page + 1):
        print(f"[INFO] {p} 페이지 수집 중...")
        page_data = crawl_music_list(p)

        # 만약 특정 페이지에 데이터가 하나도 없다면,
        # 이후 페이지도 없다고 가정하고 멈추는 방법도 가능
        # if not page_data:
        #     print("[WARN] 더 이상 데이터가 없으므로 조기 중단합니다.")
        #     break

        all_data.extend(page_data)

        # 크롤링 매너: 너무 빠른 요청은 서버에 부담이 될 수 있으므로 잠시 sleep
        time.sleep(1)

    return all_data

In [21]:
print(get_last_page_number())

5584
